In pratica è una falsa credenza che le AI possano veramente prevedere l'andamento del mercato, solo chi non se ne intende ci crede o chi ha creato codice "falso" e lo spaccia come in grado di farlo ma in realtà non è uno strumento così magico

In [ ]:
# !pip install -q tensorflow-gpu==2.0.0-beta1

try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

# More imports
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# yes, you can read dataframes from URLs!
df = pd.read_csv('https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/tf2.0/sbux.csv')

df.head()

#l'andamento è giornaliero ed è quindi un valore che varia, una time series, ci sono 4 valori
#che sono l'andamento durante la giornata in 4 momenti, il volume è il numero di dividendi delle azioni (?)
#NB dobbiamo riscalare i valori comunque e occhio che volume è su una scala diversa

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.11.0


,date,open,high,low,close,volume,Name
0,2013-02-08,27.920,28.325,27.920,28.185,7146296,SBUX
1,2013-02-11,28.260,28.260,27.930,28.070,5457354,SBUX
2,2013-02-12,28.000,28.275,27.975,28.130,8665592,SBUX
3,2013-02-13,28.230,28.230,27.750,27.915,7022056,SBUX
4,2013-02-14,27.765,27.905,27.675,27.775,8899188,SBUX


In [ ]:
df.tail()

,date,open,high,low,close,volume,Name
1254,2018-02-01,56.280,56.42,55.89,56.00,14690146,SBUX
1255,2018-02-02,55.900,56.32,55.70,55.77,15358909,SBUX
1256,2018-02-05,55.530,56.26,54.57,54.69,16059955,SBUX
1257,2018-02-06,53.685,56.06,53.56,55.61,17415065,SBUX
1258,2018-02-07,55.080,55.43,54.44,54.46,13927022,SBUX


**Adesso fa vedere come NON SI FA che è probabilmente cosa fanno vedere online spesso**





In [ ]:

# Start by doing the WRONG thing - trying to predict the price itself
series = df['close'].values.reshape(-1, 1)

# Normalize the data
# Note: I didn't think about where the true boundary is, this is just approx.
scaler = StandardScaler()
scaler.fit(series[:len(series) // 2])
series = scaler.transform(series).flatten()

### build the dataset
# let's see if we can use T past values to predict the next value
T = 10
D = 1
X = []
Y = []
for t in range(len(series) - T):
  x = series[t:t+T]
  X.append(x)
  y = series[t+T]
  Y.append(y)

X = np.array(X).reshape(-1, T, 1) # Now the data should be N x T x D
Y = np.array(Y)
N = len(X)
print("X.shape", X.shape, "Y.shape", Y.shape)


X.shape (1249, 10, 1) Y.shape (1249,)


In [ ]:


### try autoregressive RNN model
i = Input(shape=(T, 1))
x = LSTM(5)(i)
x = Dense(1)(x)
model = Model(i, x)
model.compile(
  loss='mse',
  optimizer=Adam(lr=0.1),
)

# train the RNN
r = model.fit(
  X[:-N//2], Y[:-N//2],
  epochs=80,
  validation_data=(X[-N//2:], Y[-N//2:]),
)


In [ ]:

# Plot loss per iteration
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()



In [ ]:

# One-step forecast using true targets
outputs = model.predict(X)
print(outputs.shape)
predictions = outputs[:,0]

plt.plot(Y, label='targets')
plt.plot(predictions, label='predictions')
plt.legend()
plt.show()


Una single step forecast ti riesce ma non è utile per prevedere il mercato azionario, servirebbe un multi step forecast ma come si vede sotto, con appena 2 o 3 step la previione va a puttane

In [ ]:

# Multi-step forecast
validation_target = Y[-N//2:]
validation_predictions = []

# first validation input
last_x = X[-N//2] # 1-D array of length T

while len(validation_predictions) < len(validation_target):
  p = model.predict(last_x.reshape(1, T, 1))[0,0] # 1x1 array -> scalar
  
  # update the predictions list
  validation_predictions.append(p)
  
  # make the new input
  last_x = np.roll(last_x, -1)
  last_x[-1] = p

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()


Si ottiene una linea orizzontale in pratica perchè si ripete il valore precedente.
I valori sono troppo poco correlati

In pratica stock prices non sono mai valutati con one-step.
Si usa la stock return:

### R = $ \frac{V_{final}-V_{initial}}{V_{initial}}$

Il return è dato dal prezzo finale - iniziale / iniziale su un certo lasso di tempo.


In [ ]:
# calculate returns by first shifting the data
#So first we need to shift at the closing price up by 1 so that yesterday's 
#closing price is aligned with today's closing price:

df['PrevClose'] = df['close'].shift(1) # move everything up 1

# so now it's like
# close / prev close
# x[2] x[1]
# x[3] x[2]
# x[4] x[3]
# ...
# x[t] x[t-1]



In [ ]:
df.head()

#Now the first raw is NaN because we don't have this value BTW we use the following values

In [ ]:
# then the return caluclations:

# (x[t] - x[t-1]) / x[t-1]
df['Return'] = (df['close'] - df['PrevClose']) / df['PrevClose']


df.head()

In [ ]:
# Now let's try an LSTM to predict returns
df['Return'].hist()

# Next we draw a histogram so we can see the distribution of returns we can see 
# that these are pretty small values. So we might want to normalize them

In [ ]:
series = df['Return'].values[1:].reshape(-1, 1) #from 1 since the value 0 is NaN

# Normalize the data
# Note: I didn't think about where the true boundary is, this is just approx.
scaler = StandardScaler()
scaler.fit(series[:len(series) // 2])
series = scaler.transform(series).flatten() #otteniamo un vettor lungo N


In [ ]:
### build the dataset
# let's see if we can use T past values to predict the next value
T = 10
D = 1
X = []
Y = []
for t in range(len(series) - T): #now we are working with the series as the
                                 #return rather than the actual stock price
  x = series[t:t+T]
  X.append(x)
  y = series[t+T]
  Y.append(y)

X = np.array(X).reshape(-1, T, 1) # Now the data should be N x T x D
Y = np.array(Y)
N = len(X)
print("X.shape", X.shape, "Y.shape", Y.shape)

In [ ]:
### try autoregressive RNN model
i = Input(shape=(T, 1))
x = LSTM(5)(i)
x = Dense(1)(x)
model = Model(i, x)
model.compile(
  loss='mse',
  optimizer=Adam(lr=0.01),
)

# train the RNN
r = model.fit(
  X[:-N//2], Y[:-N//2],
  epochs=80,
  validation_data=(X[-N//2:], Y[-N//2:]),
)


In [ ]:
# Plot loss per iteration
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()


In [ ]:
# One-step forecast using true targets
outputs = model.predict(X)
print(outputs.shape)
predictions = outputs[:,0]

plt.plot(Y, label='targets')
plt.plot(predictions, label='predictions')
plt.legend()
plt.show()

In [ ]:
# Multi-step forecast
validation_target = Y[-N//2:]
validation_predictions = []

# first validation input
last_x = X[-N//2] # 1-D array of length T

while len(validation_predictions) < len(validation_target):
  p = model.predict(last_x.reshape(1, T, 1))[0,0] # 1x1 array -> scalar
  
  # update the predictions list
  validation_predictions.append(p)
  
  # make the new input
  last_x = np.roll(last_x, -1)
  last_x[-1] = p

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()


Again We have the situation where the model isn't really able to do much except copy the same value over and over again.
Finally we're going to move on to the next example where we see how to actually build this model right.

By the way there isn't really anything wrong with the second model.
It just wasn't a good model.
This is unlike the first model where there actually were things wrong with it because it gave us misleading results and also predicts the wrong thing.

And this final example we are going to make use of all the data open close high and low prices as well as the volume.

Now we have TxD input T=10 D=5
Instead **we turned this into the simplest problem possible.**
Just try to predict whether the price will go up or down **In other words binary classification.**
**Regression is in general** a harder problem than classification with regression.
**You have to predict a continuous value**.
You can't be too high or too low.
You have to be right on the nose. Classification is easier especially binary classification.
You just have to predict a label.

**In this case you only have two choices up or down!**

In [ ]:
# Now turn the full data into numpy arrays

# Not yet in the final "X" format!
input_data = df[['open', 'high', 'low', 'close', 'volume']].values
targets = df['Return'].values

# Now make the actual data which will go into the neural network
T = 10 # the number of time steps to look at to make a prediction for the next day
D = input_data.shape[1] #The number of columns in the input data 
N = len(input_data) - T # (e.g. if T=10 and you have 11 data points then you'd only have 1 sample)

# normalize the inputs
Ntrain = len(input_data) * 2 // 3 #primi due terzi il test set e il validation è 1/3
scaler = StandardScaler()
scaler.fit(input_data[:Ntrain + T - 1])
input_data = scaler.transform(input_data)

# Setup X_train and Y_train
X_train = np.zeros((Ntrain, T, D))
Y_train = np.zeros(Ntrain)

for t in range(Ntrain):
  X_train[t, :, :] = input_data[t:t+T]
  Y_train[t] = (targets[t+T] > 0) #perchè >0 boh

# Setup X_test and Y_test
X_test = np.zeros((N - Ntrain, T, D))
Y_test = np.zeros(N - Ntrain)

for u in range(N - Ntrain):         #per il test abbiamo da mettere un offset Ntrain
  # u counts from 0...(N - Ntrain)
  # t counts from Ntrain...N
  t = u + Ntrain
  X_test[u, :, :] = input_data[t:t+T]
  Y_test[u] = (targets[t+T] > 0)

# make the RNN
i = Input(shape=(T, D))
x = LSTM(50)(i)
x = Dense(1, activation='sigmoid')(x)
model = Model(i, x)
model.compile(
  loss='binary_crossentropy',
  optimizer=Adam(lr=0.001),
  metrics=['accuracy'],
)

# train the RNN
r = model.fit(
  X_train, Y_train,
  batch_size=32,
  epochs=300,
  validation_data=(X_test, Y_test),

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()


We see that the train loss goes down a little bit but the validation loss goes up a lot.
So what this tells us is that the model is just again over fitting to the noise in the train set, so that's what it means when the train lost goes down.
But the validation loss goes up

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='accuracy')
plt.plot(r.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()


if we look at the accuracy we see the same thing.
The train accuracy goes up but the test accuracy stays at 50 percent.
Now you might ask why doesn't the test accuracy go down.

One important thing to realize about binary classification is that zero accuracy is not the worst accuracy if your accuracy is zero.

That means you can just reverse all your predictions and you would get one hundred percent.
That would be a perfect model.

In other words for binary classification the worst accuracy is actually 50 percent meaning it's no better
than random guessing.